In [9]:
# standard libs
import hashlib
import os
import shutil
import stat
import argparse
import re
import sqlite3
from datetime import datetime

In [10]:
# third party dependencies
import filetype

In [11]:
# my libs
import hashstore

In [12]:
BLOCKLENGTH = 262144

In [13]:
class HashStash():
    def __init__(self, db_path):
        self.db_path = db_path
        if not os.path.exists(db_path):
            print('no file at {}. '
                  'creating a new database here.'.format(db_path))
            self.init_db()
        else:
            self.conn = sqlite3.connect(db_path)
            
    def __del__(self):
        self.conn.close()
            
    def init_db(self):
        self.conn = sqlite3.connect(self.db_path)
        c = self.conn.cursor()
        c.execute('CREATE TABLE Data ('
                  'hash TEXT NOT NULL, '
                  'num_bytes INT, '
                  'media_type TEXT, '
                  'title TEXT, '
                  'PRIMARY KEY (hash) );')
        self.conn.commit()
        c.execute('CREATE TABLE HashStore ('
                  'id INTEGER NOT NULL PRIMARY KEY, '
                  'name TEXT UNIQUE, '
                  'root_path TEXT UNIQUE );')
        self.conn.commit()
        c.execute('CREATE TABLE Copy ('
                  'hash TEXT NOT NULL, '
                  'store_id INT, '
                  'FOREIGN KEY(hash) REFERENCES Data(hash), '
                  'FOREIGN KEY(store_id) REFERENCES HashStore(id), '
                  'UNIQUE(hash, store_id) );')
        self.conn.commit()
        c.execute('CREATE TABLE Source ('
                  'hash TEXT NOT NULL, '
                  'description TEXT NOT NULL, '
                  'mtime_ns INT, '
                  'ctime_ns INT, '
                  'hashing_started INT NOT NULL, '
                  'hashing_completed INT NOT NULL, '
                  'FOREIGN KEY (hash) REFERENCES Data(hash) );')
        self.conn.commit()
        del c
        
    def add_store(self, root_path, name):
        c = self.conn.cursor()
        c.execute('INSERT INTO HashStore (name, root_path) '
                  'VALUES (?, ?)', (name, root_path))
        self.conn.commit()
        del c
        
    def get_store_info(self, store_name):
        c = self.conn.cursor()
        c.execute('SELECT id, root_path FROM HashStore '
                  'WHERE name = ?', (store_name,))
        rows = c.fetchall()
        if len(rows) == 0:
            print('no store with name {}'.format(store_name))
            return None
        if len(rows) > 1:
            print('unexpected: more than one '
                  'store with name {}'.format(store_name))
            return None
        return rows[0] #(id, root_path)
        
    def add_file(self, file_path,
                 store_name='default', move=True):
        store_id, store_root_path = self.get_store_info(store_name)
        path = os.path.abspath(file_path)
        name = os.path.basename(path)
        
        mtime = os.stat(path).st_mtime_ns
        ctime = os.stat(path).st_ctime_ns
        
        if move:
            # move the file in
            hash_rec = hashstore.move_file(store_root_path, file_path)
        else:
            # copy mode
            hash_rec = hashstore.open_file(store_root_path, name)
            with open(path, 'rb') as f:
                datablock = f.read(BLOCKLENGTH)
                while len(datablock) > 0:
                    hashstore.write_to_file(hash_rec, datablock)
                    datablock = f.read(BLOCKLENGTH)
        
            mtime_2 = os.stat(path).st_mtime_ns
            if mtime_2 != mtime:
                # file modified while we were hashing it
                # it is possible that this method doesn't
                # detect modification
                # e.g. mod1...start hash...mod2...end hash
                # but time(mod1) = time(mod2) at the available
                # resolution - which is one second for some FS
                print('ABORTED. The file at {} '
                      'was modified while we were '
                      'computing its hash'.format(path))
                hashstore.abort_file(hash_rec)
                return None
            else:
                hashstore.save_file(hash_rec)
        
        sha256 = hash_rec['sha256']
            
        c = self.conn.cursor()
        c.execute('SELECT hash FROM Data '
                  'WHERE hash = ?',
                  (sha256,))
        rows = c.fetchall()
        if len(rows) == 0:
            # the hash is not currently in our DB

            # get media type
            p = hashstore.hash2filepath(store_root_path, sha256)
            media_type = filetype.guess_mime(p)

            c.execute('INSERT INTO Data '
                      '(hash, num_bytes, media_type, title) '
                      'VALUES (?, ?, ?, ?);',
                      (sha256, hash_rec['bytes_hashed'],
                       media_type, name))

        c.execute('SELECT hash FROM Copy '
                  'WHERE hash = ? AND store_id = ?',
                  (sha256, store_id))
        rows = c.fetchall()
        if len(rows) == 0:
            # DB has no record of this copy
            c.execute('INSERT INTO Copy (hash, store_id) '
                      'VALUES (?, ?);',
                      (sha256, store_id))

        c.execute('INSERT INTO Source '
                  '(hash, description,'
                  ' mtime_ns, ctime_ns,'
                  ' hashing_started,'
                  ' hashing_completed) '
                  'VALUES (?, ?, ?, ?, ?, ?)',
                  (sha256, path,
                   mtime, ctime,
                   hash_rec['hashing_started'],
                   hash_rec['hashing_completed']))
        self.conn.commit()
        del c
        return sha256
        
    def delete_copy(self, hsh, store_name):
        store_id, store_root_path = self.get_store_info(store_name)
        
        c = self.conn.cursor()
        c.execute('DELETE FROM Copy WHERE hash = ? AND store_id = ?',
                  (hsh, store_id))
        self.conn.commit()
        del c
        hashstore.delete_file(store_root_path, hsh)
        
    def delete_hash(self, hsh):
        c = self.conn.cursor()
        c.execute('SELECT '
                  'Copy.store_id, HashStore.name, HashStore.root_path '
                  'FROM Copy JOIN HashStore '
                  'ON Copy.store_id = HashStore.id '
                  'WHERE Copy.hash = ?', (hsh,))
        rows = c.fetchall()
        if len(rows) > 0:
            print('There are copies of this hash the stores:')
            for r in rows:
                print('{} (id:{}) @ {}'.format(r[1],r[0],r[2]))
            return False
        c.execute('DELETE FROM Data WHERE hash = ?', (hsh,))
        self.conn.commit()
        del c
        return True
        
    def check_integrity_of_store(self, store_name):
        store_id, store_root_path = self.get_store_info(store_name)
        
        c = self.conn.cursor()
        c.execute('SELECT hash FROM Copy WHERE store_id = ?',
                  (store_id,))
        REC_LIMIT = 1000
        row_block = c.fetchmany(REC_LIMIT)
        i = 0
        while len(row_block) > 0:
            print(i)
            i = i+1
            for r in row_block:
                is_good = hashstore.check_file(store_root_path, r[0])
                if not is_good:
                    print('bad store of {}'.format(r[0]))
            row_block = c.fetchmany(REC_LIMIT)

In [14]:
hs = HashStash('/media/wdm/magwitch/hashstash/md.db')

In [7]:
#hs.add_store('/media/wdm/magwitch/hashstash/data', 'default')

In [19]:
import pandas as pd

In [10]:
pd.read_sql('SELECT * FROM HashStore', hs.conn)

,id,name,root_path
0,1,default,/media/wdm/magwitch/hashstash/data


In [44]:
pd.read_sql('SELECT COUNT(*) FROM Source WHERE description LIKE "/media/wdm/magwitch/databag/%"', hs.conn)

,COUNT(*)
0,24114


In [38]:
pd.read_sql('SELECT media_type, count(hash) FROM Data GROUP BY media_type', hs.conn)

,media_type,count(hash)
0,None,1484
1,application/pdf,3
2,application/rtf,2
3,application/x-rar-compressed,51
4,application/x-sqlite3,2
5,audio/m4a,1494
6,audio/mpeg,9069
7,audio/ogg,4
8,audio/x-flac,2031
9,audio/x-wav,397


In [39]:
pd.read_sql('SELECT sum(num_bytes) FROM Data', hs.conn)

,sum(num_bytes)
0,334270748808


In [ ]:
pd.read_sql('SELECT media_type, count(hash) FROM Data GROUP BY media_type', hs.conn)

In [27]:
pd.read_sql('SELECT * FROM Data WHERE title LIKE "0A6%"', hs.conn)

,hash,num_bytes,media_type,title
0,d0e971af368e911028ea4c055c04cded237d7c9fdcffc0...,853635,image/jpeg,0A6B2774-0D3D-4269-94EC-ED94A91BC206.jpeg
1,181524f18184a2350c044b1c1ad6630012ae2b6972734c...,286474,image/jpeg,0A6D87CC-3FAD-4D6F-9262-A58BF94E4137.jpeg


In [35]:
pd.read_sql('SELECT * FROM Data WHERE hash LIKE "202c%"', hs.conn)

,hash,num_bytes,media_type,title
0,202c7b5f62a149dd023be9411dabdcc5fcb59cfd952fa1...,4297706,image/jpeg,67164C51-7CA7-40F6-8641-3891AFE7762E.jpeg
1,202cf35d8f2779dccf53a6d425631cc98ad3a0b43c7968...,33192431,video/mp4,486E828F-4CDF-4579-ACE0-253099DE277C.mp4


In [28]:
h = hashstore.hash_file('/media/wdm/magwitch/originals/0/'
                        '0A65DAF2-0DAF-458E-B7BB-F9CB61F36A22.mp4')

In [29]:
h

{'hashing_started': 1614291224929119232,
 'sha256': '202cf35d8f2779dccf53a6d425631cc98ad3a0b43c796831b41c78ba4e96c3c3',
 'hashing_completed': 1614291225259076096,
 'bytes_hashed': 33192431}

In [42]:
to_del = pd.read_sql('SELECT hash FROM Data WHERE media_type IS NULL', hs.conn)['hash']

In [44]:
for hsh in to_del:
    hs.delete_copy(hsh, 'default')
    hs.delete_hash(hsh)

In [12]:
f = ('/media/wdm/magwitch/hashstash/data/1b/5d/15/53/'
'f90e30e3e26cdeac3d4578a06bc364e29e2a367b17dc891a72b1fe2b/._.DS_Store')
os.path.isfile(f)

True

In [28]:
#hs.check_integrity_of_store('default')

In [21]:
def file_recurse(path, func, *args, **kwargs):
    for dirname, dirs, files in os.walk(path):
        for file in files:
            if not file.startswith('.'):
                fp = os.path.join(dirname, file)
                if command_args.verbose:
                    print(fp)
                func(fp, *args, **kwargs)

In [17]:
hs.add_file('/media/wdm/magwitch/originals/0/'
            '09827029-0CA8-4E09-9A04-D408D15D69DE.jpeg', move=True)

'229e213ee829c9eddec7b608e9e3d04a061a51d31ee48da01f865fcd8bcf92f7'

In [ ]:
# lost file - file this in the stash!
# 09997552-1EF0-4678-A86D-00E70BBBB6B1

In [36]:
command_args = lambda : None
command_args.verbose = True

file_recurse('/media/wdm/magwitch/databag',
             hs.add_file, move=True)

/media/wdm/magwitch/databag/databag/47/47/c0/34/b15f22975d88f124d677fcc7294ed152ff2834482d905bd5eba58ab3/file.mp3
/media/wdm/magwitch/databag/databag/47/09/aa/e2/1a3cf9f6f707633e4ef0b14a44b0598bfe7c4acbfdb8cc69ca647670/file.jpeg
/media/wdm/magwitch/databag/databag/47/09/18/fe/29c5a110e559888a572d9d2e79278bdd020eeae30174c01706b82593/file.data
/media/wdm/magwitch/databag/databag/47/e6/db/bf/4681368b8e1de2901922af76eb672f08a0fd545e8a9cc14ac454731c/file.jpeg
/media/wdm/magwitch/databag/databag/47/e6/b9/00/8f55922c004be37ddc400a57da9e020f76a5061edfb09d40db4c3327/file.jpeg
/media/wdm/magwitch/databag/databag/47/c6/30/b0/021198112d907a908a7664f9d1d167271d38558262809f0795f365a8/file.jpeg
/media/wdm/magwitch/databag/databag/47/ec/9c/4c/b5cf67008eb3ca9032043df27c396d4907b6166fcf71ce14a9fa35c2/file.jpeg
/media/wdm/magwitch/databag/databag/47/c5/f0/e4/8dbc5011296c15e969c82d5047e0e8a20e7dd7f5e029f37ea049c5c3/file.jpeg
/media/wdm/magwitch/databag/databag/47/8d/f4/f5/46da47fd6a8085caa33fa4df38d081e33

/media/wdm/magwitch/databag/databag/47/42/e9/fd/514ed7d74234bd259065a476e9806f7b6d6796332a40d9d8a18bebf3/file.jpeg
/media/wdm/magwitch/databag/databag/47/1b/31/46/af5221028fa0526c2f047ce92163350647a83333a8d092719fe33948/file.jpeg
/media/wdm/magwitch/databag/databag/47/ae/b4/04/1839773b0560a60ff1c8dcbb9e91ec50c57f16522b58e517550078b0/file.flv
/media/wdm/magwitch/databag/databag/47/bc/6a/02/9ab79a14c7562db25a986ded8732a8fe2ee2bcef640b939d04c9f27a/file.mp3
/media/wdm/magwitch/databag/databag/47/08/55/62/5a0524f945369bdf1287242a2698f9bb84008d782d7237a6361e7b78/file.jpeg
/media/wdm/magwitch/databag/databag/47/6e/12/8f/fe6b2a82b9a2a2a7ac7a06e396fc81a1b07fe5cddfd71bf3cdd8d792/file.jpeg
/media/wdm/magwitch/databag/databag/47/6e/a9/f1/b9c62a565cbeddfa3325902297278863e8900df2beb6e5c557d682c3/file.aif
/media/wdm/magwitch/databag/databag/47/6e/43/85/6b56d69fffdc272be70b6b3b29317c1be55a29007ed96f5329020bb4/file.mp3
/media/wdm/magwitch/databag/databag/47/3b/32/09/717afd59a0784eaf9dba45c6a287a1b4c179

/media/wdm/magwitch/databag/databag/09/46/89/b2/39d93ac345287a47975d8787a04d6db33dd5f8343b9b8ce28a38fdf6/file.mp3
/media/wdm/magwitch/databag/databag/09/f9/fc/3a/95e808a1359a941132dcafd0f2dd84525bcbc318c33108e12ef3a861/file.jpeg
/media/wdm/magwitch/databag/databag/09/66/84/25/d1b7879e3b090bca83201af4bc29e1436fd281e94d75c30eb1246a28/file.mp3
/media/wdm/magwitch/databag/databag/09/65/58/d1/e5c294ddf0df8d6d6f84a3a1eab1c69eec7520936617d25728f23b10/file.jpeg
/media/wdm/magwitch/databag/databag/09/fa/3d/79/35774d4154f445d4b5246c611a6ead25c3efbe9f6a626ba0817705ff/file.mp3
/media/wdm/magwitch/databag/databag/09/55/4b/8b/6c525b3f224597e9639ce0f91c5a888773935e6cdd09c4e7ab099db6/file.jpeg
/media/wdm/magwitch/databag/databag/09/1e/87/02/781da03840f08ca2ea665710c2db21c69570f32ba8c66ffd03c5c4e8/file.m4a
/media/wdm/magwitch/databag/databag/09/b1/e3/18/8ff12702495648bebdf4e748c24720bbcb61d169520bacb441cbca01/file.jpeg
/media/wdm/magwitch/databag/databag/09/a8/92/69/cfef971606eecd64fda5623f38f11b1a12df

/media/wdm/magwitch/databag/databag/e6/a7/ee/06/66bb94c930ff6bd28cda3818d5e3360513e727cdd09e2d7e9cee136b/file.avi
/media/wdm/magwitch/databag/databag/e6/54/00/58/2b464f95a416d5a51b121314cc63aca2f155b255086edaee63cf2404/file.mp3
/media/wdm/magwitch/databag/databag/e6/e5/39/4f/3aeca1758bd231e982c81278ed3077a0bacdff56d11c89bcbda7b4a6/file.jpeg
/media/wdm/magwitch/databag/databag/e6/2a/fa/e3/1ee3078e14bfbb36efb15bfaaa5c9d6ea63085f43c6b1d038d952574/file.mp3
/media/wdm/magwitch/databag/databag/e6/fc/cb/a5/56d603780cfe976d74aad52a7212607529dc7dd322a042aaf5495071/file.mp3
/media/wdm/magwitch/databag/databag/e6/fc/37/a0/e22f6c5ae734a4721bd8a665662b816c53448ea33e21b444fba22b74/file.mp3
/media/wdm/magwitch/databag/databag/e6/06/62/06/f727f3b2e1b4020eb1f5e2898f890660b6a8f742a29591d696006c19/file.jpeg
/media/wdm/magwitch/databag/databag/e6/d2/cd/68/71c7286a66e4935dc45b4ec1721e6512a6dc69bd35264771bfa5c34e/file.jpeg
/media/wdm/magwitch/databag/databag/e6/d2/44/8c/be9880cf2ccce07b028e7320ecc2f6694a96e

/media/wdm/magwitch/databag/databag/e6/85/14/91/307bfcd8d4067c4e801f9603a0833486f9226b1409da24ded794cae2/file.mp4
/media/wdm/magwitch/databag/databag/e6/41/29/ec/86da7c5dc4bbac51aea8c184288ccedcbf92076d90934dd7b954d2f5/file.mp3
/media/wdm/magwitch/databag/databag/e6/41/0f/80/0bf4887dc1642a53fb4a6dc18c1905bd226386c106f089093b558687/file.jpeg
/media/wdm/magwitch/databag/databag/e6/77/ea/d0/3c027cbb5c38f13e7e81fbe9cc7de66045d1dd5f57fa30aab3d8af68/file.jpeg
/media/wdm/magwitch/databag/databag/e6/18/ac/9f/28ed228c99343fe84ccc64695072ddd356836a7ee408a1eff5a20020/file.jpeg
/media/wdm/magwitch/databag/databag/e6/00/89/ec/f4a2865249a7da60c258e981996f0d21e1b653eb51c7548c4a1dee94/file.jpeg
/media/wdm/magwitch/databag/databag/c6/e6/99/33/e44b8c9f842cea4219f712b08f9ebfa1dcb7c73d55e76ea12fe2d2b5/file.jpeg
/media/wdm/magwitch/databag/databag/c6/c5/8a/67/0d8b94fcf1027fc5dea0f61393e3318a0d733e91540f4c18e9fc2612/file.mp3
/media/wdm/magwitch/databag/databag/c6/95/a4/8e/a908119e49d50c258f5effb092fefcb6682

...

/media/wdm/magwitch/databag/databag/00/01/fe/3d/56e1dfcaef5d86dd98f773ca0532e5d1c45564820a1a7ae045eafcf6/file.mp3
/media/wdm/magwitch/databag/databag/00/62/9a/32/63c00c2ebf50453b122ab95bafad852eb10e90013a8c14f9beb8617d/file.AVI
/media/wdm/magwitch/databag/databag/00/d9/5f/f9/cd4baba29f80699602f5be19202148a3f6e1d1c2fc667f6896447d9b/file.jpeg
/media/wdm/magwitch/databag/databag/00/eb/41/5a/c4c313f42d3adb401f85e3fff31894aad492826f994963135be27273/file.mp3
/media/wdm/magwitch/databag/databag/00/8c/1f/43/da740eff6e4afabd4e5603089d34addc9e86f573545069179163ef62/file.jpeg
/media/wdm/magwitch/databag/databag/00/91/b6/28/c4c19aab7a73be6370246b7ef4d7615298752444752106bfaa846b06/file.data
/media/wdm/magwitch/databag/databag/00/b6/45/50/1e15a95ccecbd8bff92730f311a5d9ccec3d4f340b63803b8cf1a44b/file.jpeg
/media/wdm/magwitch/databag/databag/00/dc/d2/c4/3f28ebaabcfdf0176ee9d1ef37c58e537c7202489d1bbe0275acf6f8/file
/media/wdm/magwitch/databag/databag/00/1b/bc/e3/2dab3ddaa188514bb080d38a634ae8776d8b5a48

/media/wdm/magwitch/databag/databag/ed/2a/c0/46/7b34fa1a34fcc19fa79e840a0c58986eac8c3fdb3b52abfbe2441508/file.jpeg
/media/wdm/magwitch/databag/databag/ed/ce/05/ef/3284781e3d840b4a9ac520f77a0fc2fc7db13a79586226589c45472d/file.mp3
/media/wdm/magwitch/databag/databag/ed/68/bd/59/5f33aec10aaaed88e16504c6ceaf632bd1133df5a3fe95efc972b978/file.mov
/media/wdm/magwitch/databag/databag/ed/d2/83/d1/f8f3e3c00d4690ffc39fbcfdfd66420df8e82925ce0ee76e70ebfa11/file.jpeg
/media/wdm/magwitch/databag/databag/ed/a0/b6/ca/10b17ea1d75a23c42c980fff688f4f56fdbda45f6d5a95d45fdaf241/file.mp3
/media/wdm/magwitch/databag/databag/ed/a3/7d/8c/f974dbc56e98399d17913ff074cc2a72557d67f1819e2d1b8fe838e8/file.mp3
/media/wdm/magwitch/databag/databag/ed/36/a3/c9/e7cc6f567b8e832be36404d741c79d4b5d55ab767134cdc992ed3d96/file.mp3
/media/wdm/magwitch/databag/databag/ed/de/ef/27/3f52c5eb5837f7c3d4ddf106a00f17b2ca90c8d570e9729202b541e1/file.mp4
/media/wdm/magwitch/databag/databag/ed/70/6c/8d/94e2353c8799012b79bb9034a71fa7c095b2ba

/media/wdm/magwitch/databag/databag/52/e6/5b/e1/bea0c6fe9f862f7991e47dd89c5cd27c489d3016915c682d76cecd37/file.jpeg
/media/wdm/magwitch/databag/databag/52/40/5d/7c/634f6d821f69cdf98ae86b7e3a991fe6ac6215690eb2b9e670cc1ab5/file.jpeg
/media/wdm/magwitch/databag/databag/52/72/72/c1/36939980be4c836b85d0c4b969b799618ae3fc17181ae8bb0f6ca679/file.mp3
/media/wdm/magwitch/databag/databag/52/72/1f/49/7d0212831bafcf825f5a14563b6009678d5d3a82ee7c7e6d6322565f/file.jpeg
/media/wdm/magwitch/databag/databag/52/4a/ed/17/a3d06b90f356dd7cf61d4d432217f30a0a1f4ed25fb0129f9c38381e/file.mp3
/media/wdm/magwitch/databag/databag/52/39/ab/a9/c80cd976071d8d612c1125bc8383d53f4cddff312b1a089e2b1b1fbf/file.avi
/media/wdm/magwitch/databag/databag/52/c5/18/fb/8881cc2ff1a3493b2a9600df1c91aa91f3974606e852e9072c273e95/file.jpeg
/media/wdm/magwitch/databag/databag/52/b7/5f/e9/6f74fd0eff7c21e3b301f66552ec7a19d07d29d9eb118f6877b115d2/file.mp3
/media/wdm/magwitch/databag/databag/52/95/a7/45/fd3f2652ad0393295c1f2c79fdd38ce62e3d

/media/wdm/magwitch/databag/databag/52/08/12/b4/3ac6a8e3fce51160d913c172e40ff34173e1833bd038ad68f6471cc7/file.jpeg
/media/wdm/magwitch/databag/databag/52/c8/ad/7f/c8d375d3b70a3c97afdd1914e337d30582245853fd554b9ced937871/file.mp3
/media/wdm/magwitch/databag/databag/52/c8/93/e7/f42fdc46d91a937f3cfe9a0d844c5a37273c06be1e1cdd8c351cdd16/file.mp3
/media/wdm/magwitch/databag/databag/52/3b/29/60/db4b14fadd2e9e049fe6ad1a6f102c0d1ebd6f03ed9838fdab441686/file.jpeg
/media/wdm/magwitch/databag/databag/52/e3/25/86/efc55dd1f9d5e7ed4bd0d23cd20f15f637783fe6cc9b96f80c080932/file.jpeg
/media/wdm/magwitch/databag/databag/52/a1/3f/2e/397970da7d8278accad7a66cbd0a2eb1478afd54201a3f60f361101c/file.jpeg
/media/wdm/magwitch/databag/databag/52/2e/c9/ea/06aa87386ebf362144140ee41b14d428eeeadefdeb44905057a5619d/file.jpeg
/media/wdm/magwitch/databag/databag/52/2e/84/f5/8ad4ae08b8b370234aa4247bbdea9b758ffbe674f8d04c638ab52135/file.jpeg
/media/wdm/magwitch/databag/databag/52/58/f5/9b/1db75a347f8c02865e963c2814ce6b291d

/media/wdm/magwitch/databag/databag/20/fa/30/18/711d5248885406a1e746f24286e00ec3eebfffe8f10d8df0237a740a/file.aif
/media/wdm/magwitch/databag/databag/20/3d/d1/6a/b2e3411d81e06116c46a988022d81a237e4213124c01aa8b701a6e64/file.jpeg
/media/wdm/magwitch/databag/databag/20/11/70/ea/79633208cc8464f54a559f454dcf675661244fc7c6e894d384cc9d78/file.jpeg
/media/wdm/magwitch/databag/databag/20/7e/fb/14/5fad74229b4fea591dfdd97440630bdef428c3b9136fa02bcec65154/file.jpeg
/media/wdm/magwitch/databag/databag/20/fe/b9/33/63a16bba2ea5b0b2bc290c68c8417b7b631b58048906d398031c0a15/file.jpeg
/media/wdm/magwitch/databag/databag/20/1c/27/04/bf82582391c355566e138de34df201f8b9053989d3c0996ace99881e/file.mp3
/media/wdm/magwitch/databag/databag/20/ad/06/d8/9e630c58b14323e839a6c7060e16460a17f2091b976426f8251103ac/file.jpeg
/media/wdm/magwitch/databag/databag/20/64/88/3f/47e4dd9cb95ace28813c2d1ee7ece0769d8653023767cb2d37764203/file.jpeg
/media/wdm/magwitch/databag/databag/20/62/7f/d1/1239d3286b9b9647e3ebb2388c8c57802f

In [8]:
del hs

In [56]:
#!rm /media/wdm/magwitch/hashstash/md.db